In [175]:
import pandas as pd

In [176]:

data = {
    'Wire': [151, 76, 23, 76, 6, 151],
    'SN': ['S88', 'S99', 'S11', 'S110', 'S44', 'S11'],
    'x1': [1, 1, 0, 1, 1, 0],
    'x2': [1, 1, 1, 1, 0, 0],
    'x3': [0, 1, 1, 1, 0, 0],
    'x4': [1, 1, 1, 1, 1, 1]
}

ypp = pd.DataFrame(data)
ypp


,Wire,SN,x1,x2,x3,x4
0,151,S88,1,1,0,1
1,76,S99,1,1,1,1
2,23,S11,0,1,1,1
3,76,S110,1,1,1,1
4,6,S44,1,0,0,1
5,151,S11,0,0,0,1


In [177]:
data = {
    'SN1': ['S1', 'S12', 'S65', 'S102', 'S101'],
    'SN2': ['S333', 'S54', 'S111', 'S876', 'S876'],
    'SN3': ['S11', '', 'S99', 'S44', 'S88'],
    'x1': [0, 0, 1, 1, 1],
    'x2': [0, 0, 1, 0, 1],
    'x3': [0, 0, 1, 0, 0],
    'x4': [1, 1, 1, 1, 1]
}

mmsta = pd.DataFrame(data)
mmsta

,SN1,SN2,SN3,x1,x2,x3,x4
0,S1,S333,S11,0,0,0,1
1,S12,S54,,0,0,0,1
2,S65,S111,S99,1,1,1,1
3,S102,S876,S44,1,0,0,1
4,S101,S876,S88,1,1,0,1


In [ ]:
import itertools
import numpy as np

def get_idx(df_wire,matching_rows):
    indices_valides = []
    for idx in df_wire.index:
        target = df_wire.loc[idx, ['x1', 'x2', 'x3', 'x4']].values
        for r in range(1, len(matching_rows) + 1):  
            for indices in itertools.combinations(matching_rows.index, r):
                pene = matching_rows.loc[list(indices)][['x1', 'x2', 'x3', 'x4']].values
                sum_pene = np.sum(pene, axis=0)
                if np.array_equal(sum_pene, target):
                    indices_valides.append((df_wire.loc[idx,'Wire'],indices))

    #print("Indices des lignes valides :", indices_valides)
    return indices_valides

def check_duplicate():
    duplicate_materials = ypp['SN'].value_counts()
    duplicates = duplicate_materials[duplicate_materials > 1]
    sn_duplicated = duplicates.index.tolist()
    return sn_duplicated

In [179]:
material_dict = []
sn_duplicated = check_duplicate()
for sn in sn_duplicated:
    valeur_a_chercher = sn
    colonnes_trouvees = [col for col in mmsta.columns if valeur_a_chercher in mmsta[col].values][0]
    mmsta_filtred_by_sn =  mmsta[mmsta[colonnes_trouvees] == valeur_a_chercher]
    ypp_filtred_by_sn = ypp[ypp['SN'] == sn]
    indices_valides = get_idx(ypp[ypp['SN']== sn] ,mmsta[mmsta[colonnes_trouvees] == sn])
    material_dict.extend(indices_valides)
    #drop ypp_filtred_by_sn dans ypp
    ypp = ypp[~ypp['SN'].isin(sn_duplicated)]

sn_duplicated = check_duplicate()

if len(sn_duplicated) == 0:
    material_to_comp_dict = dict(zip(ypp['SN'], ypp['Wire']))
    list(material_to_comp_dict.items())[:5]    

In [180]:
list(material_to_comp_dict.items())

[('S88', 151), ('S99', 76), ('S110', 76), ('S44', 6)]

In [181]:
mmsta['Wire Internal Name'] = None
for matching in material_dict:
    mmsta.loc[list(matching[1]), 'Wire Internal Name'] = matching[0]

In [182]:
idx_1 = mmsta[mmsta['Wire Internal Name'].isnull()].index
mmsta.loc[idx_1, 'Wire Internal Name'] = mmsta.loc[idx_1, 'SN1'].map(material_to_comp_dict)

In [183]:
idx_2 = mmsta[mmsta['Wire Internal Name'].isnull()].index
mmsta.loc[idx_2, 'Wire Internal Name'] = mmsta.loc[idx_2, 'SN2'].map(material_to_comp_dict)

In [184]:
idx_3 = mmsta[mmsta['Wire Internal Name'].isnull()].index
mmsta.loc[idx_3, 'Wire Internal Name'] = mmsta.loc[idx_3, 'SN3'].map(material_to_comp_dict)

In [185]:
mmsta

,SN1,SN2,SN3,x1,x2,x3,x4,Wire Internal Name
0,S1,S333,S11,0,0,0,1,151
1,S12,S54,,0,0,0,1,NaN
2,S65,S111,S99,1,1,1,1,76.0
3,S102,S876,S44,1,0,0,1,6.0
4,S101,S876,S88,1,1,0,1,151.0
